In [17]:
from __future__ import unicode_literals, print_function, division
import numpy as np
import torch

train_data_source = "C:/Users/sdat7/home/git_commit/AI_Project/study/Ajou/[2021 Spring Class] Ajou ML by JBR/train_data.npy"
train_label_source = "C:/Users/sdat7/home/git_commit/AI_Project/study/Ajou/[2021 Spring Class] Ajou ML by JBR/train_label.npy"
data_np = np.load(train_data_source, allow_pickle=True)
label_np = np.load(train_label_source, allow_pickle=True)

all_label = ['18k','17k','16k','15k','14k','13k','12k','11k','10k','9k',
'8k','7k','6k','5k','4k','3k','2k','1k','1d','2d','3d','4d','5d','6d','7d','8d','9d','Pro']

n_label = len(all_label) # 18k ~ 1k, 1d ~ 9d, pro
print("number of label: ", n_label)
print(len(data_np))
print(len(label_np))

number of label:  28
2682
2682


In [18]:
# a = torch.tensor(data_np[0][0][0]) 
# 1. 경기 2. 턴 3. 인덱스
# a = a.view(1,361)

def gameToTensor(game):
  tensor = torch.zeros(len(game),1 ,361)
  nth_turn = 0
  n_turn = len(game)

  """for i in range(n_turn):
    turn_tensor = torch.tensor(game[i][0])
    turn_tensor = turn_tensor.view(361)
    if i == 0:
      tensor[i] = turn_tensor
    else:
      ex_turn_tensor = torch.tensor(game[i-1][0])
      ex_turn_tensor = ex_turn_tensor.view(361)

      tensor[i][0] = turn_tensor - ex_turn_tensor"""

  for turn in game:
    x = torch.tensor(turn[0])
    turn_tensor = x.clone().detach()
    turn_tensor = turn_tensor.view(361)
    """if nth_turn == 0:
      # print(turn_tensor.size())

      tensor[nth_turn] = turn_tensor
      nth_turn += 1
    else:
      tensor[nth_turn] = turn_tensor - tensor[nth_turn - 1]
      nth_turn += 1
"""
    tensor[nth_turn] = turn_tensor
    nth_turn += 1

    # print(tensor.size())

    # print(nth_turn)

    # print(turn[0].size())

  
  return tensor

t = gameToTensor(data_np[0])
t[50]

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
          0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
          0.,  0.,  1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.

In [19]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(361, n_hidden, n_label)

In [20]:
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(t[0], hidden)
print(output)

tensor([[-3.3282, -3.3662, -3.3457, -3.3452, -3.3720, -3.2700, -3.3625, -3.3690,
         -3.2648, -3.3051, -3.3705, -3.3530, -3.2923, -3.3611, -3.3227, -3.3605,
         -3.3295, -3.4014, -3.3316, -3.3228, -3.2728, -3.3490, -3.3393, -3.3044,
         -3.2772, -3.3170, -3.3558, -3.3289]], grad_fn=<LogSoftmaxBackward>)


In [21]:
def categoryFromOutput(output):
  top_n, top_i = output.topk(1) # 텐서의 가장 큰 값 및 주소
  category_i = top_i[0].item()     # 텐서에서 정수 값으로 변경
  return all_label[3], category_i

print(categoryFromOutput(output))

('15k', 8)


In [22]:
import random

def randomTrainingExample():
    n = random.randint(0,2681)
    label = label_np[n]
    game = gameToTensor(data_np[n])
    category_tensor = torch.tensor([all_label.index(label)], dtype=torch.long)
    
    return label, n, category_tensor, game

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

category = 7d / line = 1454
category = 6k / line = 1353
category = 16k / line = 2512
category = 1d / line = 207
category = 13k / line = 2193
category = 16k / line = 2534
category = 12k / line = 2083
category = 6d / line = 1321
category = 1d / line = 172
category = 13k / line = 2150


In [23]:
criterion = nn.NLLLoss()

In [24]:
learning_rate = 0.01 # 이것을 너무 높게 설정하면 발산할 수 있고, 너무 낮으면 학습이 되지 않을 수 있습니다.

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # 매개변수의 경사도에 학습률을 곱해서 그 매개변수의 값에 더합니다.
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item()

In [25]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000

# 도식화를 위한 손실 추적
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # iter 숫자, 손실, 이름, 추측 화면 출력
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # 현재 평균 손실을 전체 손실 리스트에 추가
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

5000 5% (12m 40s) 4.5215 1634 / 15k ✗ (7k)
10000 10% (25m 49s) 1.6689 575 / 15k ✗ (9d)
15000 15% (39m 33s) 0.3617 1983 / 15k ✗ (11k)
20000 20% (53m 26s) 0.1552 571 / 15k ✗ (3d)


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
plt.plot(all_losses)

In [ ]:
# 혼란 행렬에서 정확한 추측을 추적
confusion = torch.zeros(n_label, n_label)
n_confusion = 1000

# 주어진 라인의 출력 반환
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    return output

# 예시들 중에 어떤 것이 정확하게 예측되었는지 기록
for i in range(n_confusion):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output = evaluate(line_tensor)
    guess, guess_i = categoryFromOutput(output)
    category_i = all_label.index(category)
    confusion[category_i][guess_i] += 1

# 모든 행을 합계로 나누어 정규화
for i in range(n_label):
    confusion[i] = confusion[i] / confusion[i].sum()

# 도식 설정
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# 축 설정
ax.set_xticklabels([''] + all_label, rotation=90)
ax.set_yticklabels([''] + all_label)

# 모든 tick에서 레이블 지정
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()

In [ ]:
results_label = []
results_level = []
def predict(n, input_line, n_predictions=3):
    results_label.append(n)
    print('\n> %s' % n)

    with torch.no_grad():
        output = evaluate(gameToTensor(input_line))

        # Get top N categories
        topv, topi = output.topk(n_predictions, 1, True)
        predictions = []

        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            results_level.append(all_label[category_index])
            print('(%.2f) %s' % (value, all_label[category_index]))
            predictions.append([value, all_label[category_index]])

In [ ]:
n_games = len(test_np)
for i in range(50):
    x =random.randint(0, 2681)
    f = gameToTensor(data_np[x])
    predict(x, f, 1)

with open("label.txt", 'w') as l:
    z = 0
    for x in results_label:
        l.write(f"{x},{results_level[z]} ")
        z+=1

print(len(results_label),len(results_level))
